In [33]:
# Dependencies
import json
import os
import pandas as pd

In [34]:
#open json
object_path = 'sample_data.json'
with open(object_path) as jsonfile:
       data_json = json.load(jsonfile)
df_from_json = pd.DataFrame(data_json)

In [35]:
df_from_json

,NCTId,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,CollaboratorClass,ConditionAncestorId,ConditionBrowseBranchAbbrev,ConditionBrowseLeafId,ConditionMeshId,...,InterventionType,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,OverallStatus,PatientRegistry,Phase,SamplingMethod,StdAge,StudyType
0,NCT04371159,Device: Velieve U.S.,Experimental,0,0,0,"[D000007239, D000014570]","[BC01, All, BXS]","[M8866, M4951, M15885, M15902]",D000014552,...,Device,Yes,No,INDUSTRY,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
1,NCT04474496,Other: Assessing impact of COVID19,0,0,0,OTHER,0,"[BC18, BC19, All]","[M5698, M5702]",0,...,Other,No,No,OTHER,Enrolling by invitation,No,0,Non-Probability Sample,"[Adult, Older Adult]",Observational
2,NCT02801266,Behavioral: Evidence informed birth control co...,"[Experimental, No Intervention]",0,0,OTHER,0,0,0,0,...,Behavioral,0,0,OTHER,Completed,0,Not Applicable,0,Adult,Interventional
3,NCT01599390,Other: Data collection,0,0,0,INDUSTRY,"[D000009976, D000012327, D000014777, D00001214...","[BC02, BC08, All, BC01]","[M8878, M8866, M4951, M11485, M16105, M13732, ...",D000007251,...,Other,0,0,INDUSTRY,Completed,0,0,Non-Probability Sample,"[Child, Adult, Older Adult]",Observational
4,NCT00312702,[Biological: Falciparum Malaria Protein 11 wit...,"[Experimental, Experimental]",Standard Deviation,"[Mean, Count of Participants, Count of Partici...","[INDUSTRY, OTHER, FED]","[D000011528, D000010272]","[BC03, All, BC01, Rare]","[M17716, M9863, M8866, M4951, M12971, M11768, ...","[D000008288, D000016778]",...,Biological,0,0,FED,Completed,0,"[Phase 1, Phase 2]",0,Adult,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NCT03768310,"[Biological: CD19.CAR-multiVST for Group A, Bi...","[Experimental, Experimental]",0,0,"[OTHER, OTHER]","[D000009370, D000009369, D000008232, D00000820...","[BC04, All, BC15, BC20, Rare]","[M9528, M9803, M26172, M9534, M10898, M9808, M...",D000054198,...,"[Biological, Biological]",No,Yes,OTHER,Not yet recruiting,0,Phase 1,0,"[Child, Adult, Older Adult]",Interventional
9996,NCT03768206,"[Behavioral: Standard Physical Therapy, Behavi...","[Experimental, Active Comparator]",0,0,OTHER,0,0,0,0,...,"[Behavioral, Behavioral, Behavioral]",No,No,OTHER,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
9997,NCT03767738,Drug: Intravitreal Aflibercept Injection (IAI),Experimental,0,0,0,D000002318,"[BC14, All]",M15983,D000014652,...,Drug,No,Yes,INDUSTRY,Completed,0,Phase 4,0,"[Adult, Older Adult]",Interventional
9998,NCT03767595,Device: ProACT Adjustable Continence Therapy f...,Experimental,0,0,0,"[D000014555, D000014570, D000059411, D000020924]","[BXS, BC23, All, BXM]","[M15882, M6519, M15883, M15888, M15902, M28051...","[D000014549, D000014550]",...,Device,Yes,No,INDUSTRY,Recruiting,0,Not Applicable,0,"[Adult, Older Adult]",Interventional


In [36]:
def explodeTable(df_name, column_name):
    tb = df_name[['NCTId', column_name]].copy()
    explode_tb = tb.explode(column_name)
    explode_tb = explode_tb.replace(0, "-")
    explode_tb = explode_tb.sort_values(by=[column_name], ascending=False)
    fr = len(explode_tb) - explode_tb[column_name].str.contains('-', na=False).sum()
    print('feature_row', fr)
    feature_tb = explode_tb.head(fr)
    #.rename(columns={column_name:'features'})

    return feature_tb

In [40]:
tb = df_from_json[['NCTId', 'ArmGroupInterventionName']].copy()
explode_tb = tb.explode('ArmGroupInterventionName')
tb['feature'] = tb['ArmGroupInterventionName'].str.split(' ').str[0]
tb = tb.drop(['ArmGroupInterventionName'], axis=1)
tb = tb.rename(columns={'feature':'ArmGroupInterventionName'})
lb = tb['ArmGroupInterventionName'].unique()
lb

array(['Device:', 'Other:', 'Behavioral:', nan, 'Procedure:', 'Drug:',
       'Genetic:', 'Biological:', 'Dietary', 'Diagnostic', 'Radiation:',
       'Combination'], dtype=object)

In [5]:
tb1= explodeTable(df_from_json, 'ArmGroupInterventionName')
tb1['feature'] = tb1['ArmGroupInterventionName'].str.split(' ').str[0]
tb1 = tb1.drop(['ArmGroupInterventionName'], axis=1)
tb1 = tb1.rename(columns={'feature':'ArmGroupInterventionName'})
l1 = tb1['ArmGroupInterventionName'].unique()
l1

feature_row 17433


array(['Radiation:', 'Procedure:', 'Other:', 'Genetic:', 'Drug:',
       'Dietary', 'Diagnostic', 'Device:', 'Combination', 'Biological:',
       'Behavioral:'], dtype=object)

In [6]:
tb2= explodeTable(df_from_json, 'ArmGroupType')
l2 = tb2['ArmGroupType'].unique()
l2

feature_row 15147


array(['Sham Comparator', 'Placebo Comparator', 'Other',
       'No Intervention', 'Experimental', 'Active Comparator'],
      dtype=object)

In [7]:
tb3= explodeTable(df_from_json, 'BaselineMeasureDispersionType')
l3 = tb3['BaselineMeasureDispersionType'].unique()
l3

feature_row 2484


array(['Standard Deviation', 'Inter-Quartile Range', 'Full Range'],
      dtype=object)

In [8]:
tb4= explodeTable(df_from_json, 'BaselineMeasureParamType')
l4 = tb4['BaselineMeasureParamType'].unique()
l4

feature_row 8104


array(['Number', 'Median', 'Mean', 'Least Squares Mean', 'Geometric Mean',
       'Count of Units', 'Count of Participants'], dtype=object)

In [9]:
tb5= explodeTable(df_from_json, 'CollaboratorClass')
l5 = tb5['CollaboratorClass'].unique()
l5

feature_row 8665


array(['UNKNOWN', 'OTHER_GOV', 'OTHER', 'NIH', 'NETWORK', 'INDUSTRY',
       'FED', 'AMBIG'], dtype=object)

In [10]:
#some problem please look again
tb6= explodeTable(df_from_json, 'ConditionAncestorId')
l6 = tb6['ConditionAncestorId'].unique()
l6

In [11]:
tb7= explodeTable(df_from_json, 'ConditionBrowseBranchAbbrev')
l7 = tb7['ConditionBrowseBranchAbbrev'].unique()
l7

feature_row 30386


array(['Rare', 'BXS', 'BXM', 'BC26', 'BC25', 'BC24', 'BC23', 'BC21',
       'BC20', 'BC19', 'BC18', 'BC17', 'BC16', 'BC15', 'BC14', 'BC11',
       'BC10', 'BC09', 'BC08', 'BC07', 'BC06', 'BC05', 'BC04', 'BC03',
       'BC02', 'BC01', 'All'], dtype=object)

In [12]:
tb8= explodeTable(df_from_json, 'ConditionBrowseLeafId')
l8 = tb8['ConditionBrowseLeafId'].unique()
l8

feature_row 62916


array(['T995', 'T950', 'T949', ..., 'M10111', 'M10093', 'M10023'],
      dtype=object)

In [13]:
tb9= explodeTable(df_from_json, 'ConditionMeshId')
l9 = tb9['ConditionMeshId'].unique()
l9

feature_row 15796


array(['D000078064', 'D000077342', 'D000077277', ..., 'D000000022',
       'D000000013', 'D000000006'], dtype=object)

In [14]:
tb10= explodeTable(df_from_json, 'DesignAllocation')
l10 = tb10['DesignAllocation'].unique()
l10

feature_row 6485


array(['Randomized', 'Non-Randomized', 'N/A'], dtype=object)

In [15]:
tb11= explodeTable(df_from_json, 'DesignMasking')
l11 = tb11['DesignMasking'].unique()
l11

feature_row 7283


array(['Triple', 'Single', 'Quadruple', 'None (Open Label)', 'Double'],
      dtype=object)

In [16]:
tb12= explodeTable(df_from_json, 'DesignPrimaryPurpose')
l12 = tb12['DesignPrimaryPurpose'].unique()
l12

feature_row 7199


array(['Treatment', 'Supportive Care', 'Screening', 'Prevention', 'Other',
       'Health Services Research', 'Educational/Counseling/Training',
       'Diagnostic', 'Device Feasibility', 'Basic Science'], dtype=object)

In [17]:
tb13= explodeTable(df_from_json, 'DesignTimePerspective')
l13 = tb13['DesignTimePerspective'].unique()
l13

feature_row 1462


array(['Retrospective', 'Prospective', 'Other'], dtype=object)

In [18]:
tb14= explodeTable(df_from_json, 'EnrollmentCount')
l14 = tb14['EnrollmentCount'].unique()
l14

feature_row 9021


array(['9999', '9942', '994', ..., '10', '1', '0'], dtype=object)

In [19]:
tb15= explodeTable(df_from_json, 'HealthyVolunteers')
l15 = tb15['HealthyVolunteers'].unique()
l15

feature_row 8994


array(['No', 'Accepts Healthy Volunteers'], dtype=object)

In [20]:
tb16= explodeTable(df_from_json, 'InterventionAncestorId')
l16 = tb16['InterventionAncestorId'].unique()
l16

feature_row 21879


array(['D000078622', 'D000077264', 'D000077221', 'D000077203',
       'D000075242', 'D000074322', 'D000074268', 'D000072664',
       'D000068796', 'D000068776', 'D000068760', 'D000067856',
       'D000065701', 'D000065698', 'D000065697', 'D000065696',
       'D000065695', 'D000065694', 'D000065693', 'D000065692',
       'D000065690', 'D000065689', 'D000065688', 'D000065687',
       'D000065686', 'D000065609', 'D000065607', 'D000065427',
       'D000065171', 'D000065147', 'D000065130', 'D000065128',
       'D000065127', 'D000065108', 'D000065105', 'D000065104',
       'D000065101', 'D000065100', 'D000065098', 'D000065096',
       'D000065095', 'D000065093', 'D000065092', 'D000065088',
       'D000065087', 'D000065086', 'D000064804', 'D000064729',
       'D000064690', 'D000064449', 'D000064426', 'D000063387',
       'D000063386', 'D000063385', 'D000062865', 'D000062686',
       'D000062646', 'D000061567', 'D000059005', 'D000059004',
       'D000059003', 'D000058987', 'D000058986', 'D0000

In [21]:
tb17= explodeTable(df_from_json, 'InterventionBrowseBranchAbbrev')
l17 = tb17['InterventionBrowseBranchAbbrev'].unique()
l17

feature_row 14599


array(['Vi', 'VaDiAg', 'VaCoAg', 'Urol', 'Resp', 'Repr', 'Rena',
       'PsychDr', 'PlAggInh', 'PhSol', 'Ot', 'NootAg', 'NiOxD', 'NeuroAg',
       'NarcAntag', 'NaAg', 'MuRelCen', 'Micro', 'Mi', 'Lipd', 'Infl',
       'Infe', 'Hypo', 'Hemat', 'HB', 'Gast', 'Fl', 'FiAg', 'Emet',
       'Derm', 'Coag', 'ChanBlk', 'CaAg', 'CNSSti', 'CNSDep', 'BlSubst',
       'BDCA', 'Antipy', 'AntiConv', 'Analg', 'AnTuAg', 'AnSickAg',
       'AnObAg', 'AnEm', 'AnDyAg', 'AnCoag', 'AnArAg', 'AnAg', 'All',
       'AlcDet', 'AdjAn', 'ARhu', 'ANeo', 'AAll', 'AA'], dtype=object)

In [22]:
tb18= explodeTable(df_from_json, 'InterventionBrowseLeafId')
l18 = tb18['InterventionBrowseLeafId'].unique()
l18

feature_row 26733


array(['T94', 'T9', 'T88', ..., 'M10030', 'M10029', 'M10019'],
      dtype=object)

In [23]:
tb19= explodeTable(df_from_json, 'InterventionMeshId')
l19 = tb19['InterventionMeshId'].unique()
l19

feature_row 5598


array(['D000078862', 'D000078785', 'D000078784', ..., 'C000002854',
       'C000001205', 'C000000489'], dtype=object)

In [24]:
tb20= explodeTable(df_from_json, 'InterventionType')
l20 = tb20['InterventionType'].unique()
l20

feature_row 15695


array(['Radiation', 'Procedure', 'Other', 'Genetic', 'Drug',
       'Dietary Supplement', 'Diagnostic Test', 'Device',
       'Combination Product', 'Biological', 'Behavioral'], dtype=object)

In [25]:
tb21= explodeTable(df_from_json, 'IsFDARegulatedDevice')
l21 = tb21['IsFDARegulatedDevice'].unique()
l21

feature_row 3968


array(['Yes', 'No'], dtype=object)

In [26]:
tb22= explodeTable(df_from_json, 'IsFDARegulatedDrug')
l22 = tb22['IsFDARegulatedDrug'].unique()
l22

feature_row 3980


array(['Yes', 'No'], dtype=object)

In [27]:
tb23= explodeTable(df_from_json, 'LeadSponsorClass')
l23 = tb23['LeadSponsorClass'].unique()
l23

feature_row 9157


array(['OTHER_GOV', 'OTHER', 'NIH', 'NETWORK', 'INDUSTRY', 'INDIV', 'FED'],
      dtype=object)

In [28]:
tb25= explodeTable(df_from_json, 'PatientRegistry')
l25 = tb25['PatientRegistry'].unique()
l25

feature_row 1037


array(['Yes', 'No'], dtype=object)

In [29]:
#run into prolem need to check later
tb27= explodeTable(df_from_json, 'SamplingMethod')
l27 = tb27['SamplingMethod'].unique()
l27

In [30]:
tb28= explodeTable(df_from_json, 'StdAge')
l28 = tb28['StdAge'].unique()
l28

feature_row 17153


array(['Older Adult', 'Child', 'Adult'], dtype=object)

In [31]:
tb29= explodeTable(df_from_json, 'StudyType')
l29 = tb29['StudyType'].unique()
l29

feature_row 9157


array(['Observational', 'Interventional', 'Expanded Access'], dtype=object)